In [ ]:
from tensorflow import keras
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import pickle


# f2 = open('data/split_0.80_0.10_0.10_aug_to_5000_down_sample_True_instance1/val_data', 'rb')
# f2 = open('data/split_0.80_0.10_0.10_aug_to_5000_down_sample_True_instance1/val_data', 'rb')
# X_sequence_val, X_feature_val, X_image_val, Y_val = pickle.load(f2)
# f2.close()
# f3 = open('data/split_0.80_0.10_0.10_aug_to_5000_down_sample_True_instance1/test_data', 'rb')
f3 = open('data/split_image_0.60_0.30_0.10_aug_to_1875_down_sample_True_instance0-9/test_data', 'rb')
# f3 = open('data/split_0.60_0.30_0.10_aug_to_1875_down_sample_False_instance0-9/test_data', 'rb')
X_sequence_test, X_feature_test, X_image_test, Y_test = pickle.load(f3)
# X_sequence_test, X_feature_test, Y_test = pickle.load(f3)
f3.close()
from keras_preprocessing.sequence import pad_sequences
X_sequence_test_paded = pad_sequences(X_sequence_test, dtype='float', padding='post', value = np.nan)
# X_sequence_val_paded = pad_sequences(X_sequence_val, dtype='float', padding='post', value = np.nan)
from util import preprocess
X_sequence_test_trimed = preprocess(X_sequence_test_paded)
# X_sequence_val_trimed = preprocess(X_sequence_val_paded)

def build_channel(data):
    X = []
    for image in data:
        X.append(image.reshape(128, 128, 1))
    return X
X_image_test = np.array(build_channel(X_image_test))
# X_image_val = np.array(build_channel(X_image_val))
X_feature_test = np.array(X_feature_test)
# X_feature_val = np.array(X_feature_val)
Y_test = np.array(Y_test)-1
# Y_val = np.array(Y_val)-1

In [ ]:
models = []
best_list_0=[18,8,10,13,10,16,8,16,12,11]
best_list_1=[17,25,16,10,4,25,22,13,21,10]
best_list_2=[11,10,11,7,14,10,16,9,21,11]
best_list_3=[10,26,13,16,15,16,13,12,11,21]
best_acc_list_before_image = [6,5,4,3,5,8,5,4,7,8]
min_loss_list_before_image = [6,6,4,8,7,8,5,6,5,8]
best_acc_list_after_image = [22, 10, 51, 76, 29, 51, 33, 96, 79, 5]
min_loss_list_after_image = [55, 30, 30, 57, 87, 29, 88, 96, 85, 67]
best_acc_list_only_image = [36, 51, 57, 78, 86, 98, 80, 43, 93, 57]
min_loss_list_only_image = [39, 87, 96, 81, 86, 90, 78, 96, 97, 95]

best_acc_list_x10 = [4,3,5,6,3,6,5,7,3,8]
min_loss_list_x10 = [4,4,4,5,6,6,3,7,8,6]

for i in range(0,10):
    # lodaed_model = keras.models.load_model('models/bagging_only_image_0.60_0.30_0.10/cnn_with_feature%d/model_%d'%(i, best_acc_list_only_image[i]),compile=False)
    # lodaed_model = keras.models.load_model('models/bagging_before_image_0.60_0.30_0.10/rnn_with_feature%d/model_%d'%(i,best_acc_list_before_image[i]+2),compile=False)
    lodaed_model = keras.models.load_model('models/bagging_after_image_0.60_0.30_0.10/mix_model%d/model_%d'%(i,best_acc_list_after_image[i]),compile=False)
    # lodaed_model = keras.models.load_model('models/10x_0.60_0.30_0.10/rnn_with_feature%d_extra/model_%d'%(i,min_loss_list_x10[i]),compile=False)
    models.append(lodaed_model)

In [ ]:

# def generate_batch(X_sequence, X_image,  X_feature, Y, batch_size):
# def generate_batch(X_sequence,  X_feature, Y, batch_size):
#     idx = 0
#     batch_num = len(Y)/batch_size
#     while idx < batch_num:
#         batch_x_sequence = X_sequence[idx * batch_size : (idx + 1) * batch_size]
#         batch_x_feature = X_feature[idx * batch_size : (idx + 1) * batch_size]
#         batch_y = Y[idx * batch_size : (idx + 1) * batch_size]
#         yield batch_x_sequence,  batch_x_feature, batch_y
#         idx += 1
from util import generate_batch
test_gen = generate_batch(X_sequence_test_trimed, X_image_test, X_feature_test, Y_test, 128)      
# test_gen = generate_batch(X_sequence=X_sequence_test_trimed,X_feature=X_feature_test, Y= Y_test,batch_size=128)     
y_label = Y_test

# val_gen = generate_batch(X_sequence_val_trimed, X_image_val, X_feature_val, Y_val, 128)      
# y_label = Y_val


y_predict = np.array([])
for x_sequence, x_image, x_feature, y in test_gen:
# for x_sequence,  x_feature, y in test_gen:
    results = []
    for model in models:
    # for i in (9,):
    #     model = models[i]
        results.append(model.predict({'sequence':x_sequence, 
                                        'image':x_image,
                                        'feature':x_feature}))
    average_result = np.sum(results, axis=0)
    y_predict = np.concatenate([y_predict, np.argmax(average_result, axis=-1)])
y_label = y_label.astype(np.int32) 
y_predict = y_predict.astype(np.int32) 
cm = confusion_matrix(y_true=y_label, y_pred=y_predict, normalize=None, labels=[0,3,1,2,6,4,5,7,8,9,10])
print(cm.shape)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm)
# disp.plot(values_format='.2g')
balanced_acc = 0
for i in range(0,11):
    balanced_acc += cm[i][i]
balanced_acc = balanced_acc / 11
print('balanced_acc:',balanced_acc)

# classes = ['RRab', 'RRc','RRd', 'Blazhko','Ecl', 'EA' , 'Rot', 'LPV','δ-Scuti','ACEP', 'Cep-II']
classes = ['RRab', 'Blazhko','RRc','RRd','Rot', 'Ecl', 'EA' , 'LPV','δ-Scuti','ACEP', 'Cep-II']


from util import cm_plot

plotting = cm_plot(cm, classes_types=classes, classifier_model='mix Classifier', nClasses=11,
                    cmap=plt.cm.Blues)

In [ ]:
import pickle
f = open('y_pre_and_y_label_ensemble_after_image', 'wb')
pickle.dump((y_predict,y_label), f)

In [ ]:
import itertools

def plot_confusion_matrixs(cm1, classes_types,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Reds):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm1 = cm1.astype('float') / cm1.sum(axis=0)[np.newaxis,:] * 100
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    

    fig = plt.figure(figsize=(12,10))

    plt.imshow(cm1, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=16)
    cb=plt.colorbar(fraction=0.046, pad=0.04)
    cb.ax.tick_params(labelsize=16)
    tick_marks = np.arange(len(classes_types))
    plt.xticks(tick_marks, classes_types, rotation=45)
    plt.yticks(tick_marks, classes_types)
    plt.tick_params(axis='x', labelsize=16)
    plt.tick_params(axis='y', labelsize=16)

    thresh = cm1.max() / 2.
    for i, j in itertools.product(range(cm1.shape[0]), range(cm1.shape[1])):
        plt.text(j, i, "{:0.1f}".format(cm1[i, j]),
                 horizontalalignment="center",
                 color="white" if (cm1[i, j] < 1) or (cm1[i,j] >= 75)  else "black",fontsize=18)

    plt.ylabel('True label',fontsize = 16)
    plt.xlabel('Predicted label', fontsize = 16)
    plt.tight_layout()

   

    return fig

In [ ]:

plot_confusion_matrixs(cm, classes,
                          normalize=True,
                          title='Confusion matrix for compound multi-input neural network',
                          cmap=plt.cm.Reds)
plt.savefig('precison_after_image.svg')

In [ ]:
import pandas as pd
import numpy as np
SPE_data = pd.read_csv("Type_FL_1Misclassification_SPE.csv", sep=' ')

ture_label = np.array(SPE_data["True_class_labels"])-1
idx = np.where(ture_label==11)
ture_label[idx] = 10

pre_label = np.array(SPE_data["Prediction"])-1
idx = np.where(pre_label==11)
pre_label[idx] = 10

cm_SPE = confusion_matrix(y_true=ture_label, 
                          y_pred=pre_label, normalize='true')
                          # ,labels = [1,2,3,4,5,6,7,8,9,10,12])
classes = ['RRab', 'RRc','RRd', 'Blazhko','Ecl', 'EA' , 'Rot', 'LPV','δ-Scuti','ACEP', 'Cep-II']
plotting = cm_plot(cm_SPE, classes_types=classes, classifier_model='mix Classifier', nClasses=11,
                    cmap=plt.cm.Blues)


In [ ]:
wrong_pre_index = []
right_pre_index = []
for i in range(0,len(y_predict)):
    if y_predict[i] != y_label[i]:
        wrong_pre_index.append(i)
    else:
        right_pre_index.append(i)

In [ ]:
splited_wrong_pre_index = [[] for i in range(0,11)]
for index in wrong_pre_index:
    label = y_label[index]
    splited_wrong_pre_index[label].append(index)
splited_right_pre_index = [[] for i in range(0,11)]
for index in right_pre_index:
    label = y_label[index]
    splited_right_pre_index[label].append(index)

In [ ]:
for i ,list in enumerate(splited_wrong_pre_index):
    print('%d, length:%d'%(i, len(list)))

In [ ]:

print(len(wrong_pre_index))
index = splited_wrong_pre_index[3][2]
# index = splited_right_pre_index[1][23]
print('y_label:%d'%y_label[index])
print('y_predict:%d'%y_predict[index])
x = X_sequence_test[index][:,0]
y = X_sequence_test[index][:,1]
err = X_sequence_test[index][:,2]
plt.errorbar(x,y,err, fmt='o',ms=4, mfc='r', elinewidth=1, capsize=2)
# plt.imshow(X_image_test[index])
print(X_feature_test[index])
print('period:%f,  amplitude:%f'%(X_feature_test[index][0], X_feature_test[index][1]))

results = []
for model in models:
    results.append(model.predict({'sequence':np.array([X_sequence_test_trimed[index]]), 
                                        'image':np.array([X_image_test[index]]),
                                        'feature':np.array([X_feature_test[index]])}))
average_result = np.sum(results, axis=0) / 11
print('model result:' )
print(average_result)